In [31]:
import nltk
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

import numpy as np
from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = json.loads(open('words_data.json').read())
classes = json.loads(open('classes_data.json').read())

In [32]:
model

In [33]:
print(words)
print(classes)

['a', 'am', 'anyone', 'are', 'awesome', 'book', 'buy', 'bye', 'can', 'chatting', 'check', 'comedy', 'cost', 'day', 'do', 'doe', 'finished', 'for', 'genre', 'good', 'goodbye', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'how', 'i', 'is', 'later', 'me', 'money', 'much', 'newest', 'next', 'nice', 'novel', 'out', 'pay', 'please', 'recommend', 'romantic', 'see', 'sell', 'shopping', 'some', 'thank', 'thanks', 'that', 'the', 'there', 'this', 'till', 'time', 'to', 'want', 'well', 'you']
['buybooks', 'goodbye', 'greeting', 'noanswer', 'payment', 'thanks']


In [34]:
def clean(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
#     print(res)

    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [35]:
def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result
def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

In [48]:
# while(True):
#     w = str(input())
#     print("You: ", w)
#     res = chatbot_response(w)
#     print("Bot: ",res)
    

## Build the User Interface

In [37]:
from flask import Flask, render_template, request

In [ ]:
app = Flask(__name__)
app.config.update(
    TEMPLATES_AUTO_RELOAD = True
)
path = "main.html"

@app.route("/")
def home():
    return (render_template("main.html"))

@app.route("/get")
def bot_response():
    text = request.args.get('msg')
    return chatbot_response(text)
if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/May/2021 15:00:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:00:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:00:37] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:00:37] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [03/May/2021 15:11:20] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:11:20] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:11:20] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:11:20] "GET /style.css HTTP/1.1" 404 -
127.0.0.1 - - [03/May/2021 15:16:08] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:16:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:16:08] "GET /style.css HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:16:08] "GET /style.css HTTP/1.1" 404 -
127.0.0.

127.0.0.1 - - [03/May/2021 15:58:29] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:58:29] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:59:17] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:59:17] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:59:20] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:59:20] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 15:59:21] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 15:59:21] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:00:02] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 16:00:02] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:00:07] "GET /?msg=Hello HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 16:00:07] "GET /?msg=Hello HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 16:00:08] "GET /?msg= HTTP/1.1" 200 -
INFO:werkze

127.0.0.1 - - [03/May/2021 19:59:57] "GET /get?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20Hello%3C%2Fspan%3E%3Cbr%3E HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 19:59:57] "GET /get?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20Hello%3C%2Fspan%3E%3Cbr%3E HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2021 20:00:06] "GET /get?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20I%20want%20to%20buy%20a%20new%20toy%3C%2Fspan%3E%3Cbr%3E HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [03/May/2021 20:00:06] "GET /get?msg=%3Cspan%20class%20%3D%20%22message%20user%22%3E%20User%3A%20I%20want%20to%20buy%20a%20new%20toy%3C%2Fspan%3E%3Cbr%3E HTTP/1.1" 200 -
